In [1]:
import pandas as pd
import time

In [2]:
path_train = "../resource/PINGAN-2018-train_demo.csv"
path_test = "../resource/PINGAN-2018-test_demo.csv"

In [3]:
def load_data(path_train,path_test):
    train_data = pd.read_csv(path_train)
    test_data = pd.read_csv(path_test)
    return train_data,test_data

In [4]:
#加载数据
train_data,test_data=load_data(path_train,path_test)

In [5]:
#拼接训练集和测试集进行特征工程
test_data['TERMINALNO'] = test_data['TERMINALNO']+train_data['TERMINALNO'].max()
data = pd.concat([train_data,test_data])

In [6]:
#重置index
data.reset_index(drop=True,inplace=True)

In [7]:
#时间处理
def time_datetime(value):
    format = '%Y%m%d%H%M'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

def time_date(value):
    format = '%Y%m%d'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

def time_hour(value):
    format = '%H'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)
def time_minute(value):
    format = '%M'
    value = time.localtime(value)
    dt = time.strftime(format, value)
    return int(dt)

#转换成时刻
data['time']=data['TIME'].apply(time_datetime)
data['date']=data['TIME'].apply(time_date)
data['hour']=data['TIME'].apply(time_hour)
data['minute']=data['TIME'].apply(time_minute)

In [8]:
# trip_max
feature=pd.DataFrame()
feature[['TERMINALNO','trip_max']]=pd.DataFrame(data['TRIP_ID'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','TRIP_ID']]

# lon_max lon_min lon
lonmax=pd.DataFrame()
lonmin=pd.DataFrame()
lonmax[['TERMINALNO','lon_max']]=pd.DataFrame(data['LONGITUDE'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','LONGITUDE']]
lonmin[['TERMINALNO','lon_min']]=pd.DataFrame(data['LONGITUDE'].groupby(data['TERMINALNO']).min()).reset_index()[['TERMINALNO','LONGITUDE']]
feature=pd.merge(feature,lonmax,how='left',on='TERMINALNO')
feature=pd.merge(feature,lonmin,how='left',on='TERMINALNO')
feature['lon']=feature['lon_max']-feature['lon_min']

# lat_max lat_min lat
latmax=pd.DataFrame()
latmin=pd.DataFrame()
latmax[['TERMINALNO','lat_max']]=pd.DataFrame(data['LATITUDE'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','LATITUDE']]
latmin[['TERMINALNO','lat_min']]=pd.DataFrame(data['LATITUDE'].groupby(data['TERMINALNO']).min()).reset_index()[['TERMINALNO','LATITUDE']]
feature=pd.merge(feature,latmax,how='left',on='TERMINALNO')
feature=pd.merge(feature,latmin,how='left',on='TERMINALNO')
feature['lat']=feature['lat_max']-feature['lat_min']

# heg_max heg_min heg_mean heg
hegmax=pd.DataFrame()
hegmin=pd.DataFrame()
hegmean=pd.DataFrame()
hegmax[['TERMINALNO','heg_max']]=pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','HEIGHT']]
hegmin[['TERMINALNO','heg_min']]=pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).min()).reset_index()[['TERMINALNO','HEIGHT']]
hegmean[['TERMINALNO','heg_mean']]=pd.DataFrame(data['HEIGHT'].groupby(data['TERMINALNO']).mean()).reset_index()[['TERMINALNO','HEIGHT']]
feature=pd.merge(feature,hegmax,how='left',on='TERMINALNO')
feature=pd.merge(feature,hegmin,how='left',on='TERMINALNO')
feature=pd.merge(feature,hegmean,how='left',on='TERMINALNO')
feature['heg']=feature['heg_max']-feature['heg_min']

# volu 活动区间体积
feature['vol']=feature['lon']*feature['lat']*feature['heg']

# 速度 sp_max sp_mean
spmax=pd.DataFrame()
spmean=pd.DataFrame()
spmax[['TERMINALNO','sp_max']]=pd.DataFrame(data['SPEED'].groupby(data['TERMINALNO']).max()).reset_index()[['TERMINALNO','SPEED']]
spmean[['TERMINALNO','sp_mean']]=pd.DataFrame(data['SPEED'].groupby(data['TERMINALNO']).mean()).reset_index()[['TERMINALNO','SPEED']]
feature=pd.merge(feature,spmax,how='left',on='TERMINALNO')
feature=pd.merge(feature,spmean,how='left',on='TERMINALNO')

#callstate
call0=pd.DataFrame()
call1=pd.DataFrame()
call0[['TERMINALNO','call0']]=pd.DataFrame(data['CALLSTATE'][data['CALLSTATE'] == 0].groupby(data['TERMINALNO']).count()).reset_index()[['TERMINALNO','CALLSTATE']]
call1[['TERMINALNO','call1']]=pd.DataFrame(data['CALLSTATE'][data['CALLSTATE'] > 0].groupby(data['TERMINALNO']).count()).reset_index()[['TERMINALNO','CALLSTATE']]
feature=pd.merge(feature,call0,how='left',on='TERMINALNO')
feature=pd.merge(feature,call1,how='left',on='TERMINALNO')

feature['call0'].fillna(0,inplace=True)
feature['call1'].fillna(0,inplace=True)
feature['call_ratio_0']=feature['call0']/(feature['call0']+feature['call1'])
feature['call_ratio_1']=feature['call1']/(feature['call0']+feature['call1'])

In [66]:
from math import sqrt
# 行程
# 对每个USER 按 TIME 排序
sortdata = data.sort_values(['TERMINALNO','time']).reset_index(drop = True)
# 删除TRIP_ID后去重
del sortdata['TRIP_ID']
sortdata.drop_duplicates(inplace=True)
# 计算经纬度差
sortdata['difflat'] = sortdata.groupby(['TERMINALNO'])['LATITUDE'].diff()
sortdata['difflon'] = sortdata.groupby(['TERMINALNO'])['LONGITUDE'].diff()
# 对每个用户的第一个经纬度差置0
sortdata.fillna(0.0,inplace=True)
# 计算单个距离
sortdata['dis2'] = sortdata['difflat'] ** 2 + sortdata['difflon'] ** 2
sortdata['dis'] = sortdata['dis2'].apply(sqrt)
del sortdata['dis2']
# 计算总行程
# disdata = pd.DataFrame()
# disdata[['TERMINALNO','dis']]=sortdata['dis'].groupby(['TERMINALNO']).sum()
disdata = sortdata['dis'].groupby(sortdata['TERMINALNO']).sum().reset_index()
feature = pd.merge(feature,disdata,how='left',on='TERMINALNO')
feature

,TERMINALNO,trip_max,lon_max,lon_min,lon,lat_max,lat_min,lat,heg_max,heg_min,heg_mean,heg,vol,sp_max,sp_mean,call0,call1,call_ratio_0,call_ratio_1,dis
0,1,45,124.809792,122.971771,1.838021,43.506298,41.082325,2.423973,224.069580,-7.248444,47.848093,231.318024,1030.594264,32.779999,17.489840,144.0,150.0,0.489796,0.510204,8.182594
1,2,66,117.079720,113.330002,3.749718,30.790524,22.917767,7.872757,526.300537,-112.462982,71.400718,638.763519,18856.694241,36.119999,9.287734,289.0,439.0,0.396978,0.603022,21.248730
2,3,37,113.918510,112.899796,1.018714,28.856243,22.474741,6.381502,125.748291,-10.797699,43.942672,136.545990,887.675299,25.440001,7.987331,838.0,125.0,0.870197,0.129803,17.672516
3,4,98,116.943901,116.751045,0.192856,33.992123,33.473557,0.518566,115.885498,-14.475311,31.659250,130.360809,13.037197,33.310001,6.312753,181.0,611.0,0.228535,0.771465,3.677815
4,5,85,115.024368,114.194641,0.829727,31.188187,30.531494,0.656693,117.702576,-14.855835,29.461915,132.558411,72.227885,53.480000,7.695846,903.0,0.0,1.000000,0.000000,7.396169
5,6,40,120.920166,120.510963,0.409203,31.661022,31.277920,0.383102,130.427490,-18.174103,9.389893,148.601593,23.295750,29.570000,11.456653,478.0,0.0,1.000000,0.000000,3.459416
6,7,138,108.436508,106.474197,1.962311,30.864641,29.538458,1.326183,432.401733,162.777222,246.410570,269.624511,701.666376,35.680000,5.705086,321.0,1132.0,0.220922,0.779078,15.070974
7,8,59,122.006638,121.575500,0.431138,39.647110,38.863209,0.783901,188.175537,-13.241180,36.571975,201.416717,68.072709,42.720001,9.100117,662.0,36.0,0.948424,0.051576,4.446437
8,9,9,121.155373,120.154106,1.001267,31.774067,30.354998,1.419069,32.210754,-17.095184,5.575440,49.305938,70.057178,35.680000,16.274006,317.0,0.0,1.000000,0.000000,4.448589
9,10,57,106.610077,105.911392,0.698685,29.581587,29.359980,0.221607,400.146973,175.481934,305.556835,224.665039,34.785671,32.330002,7.297735,3.0,641.0,0.004658,0.995342,3.394046


In [10]:
data.head()

,CALLSTATE,DIRECTION,HEIGHT,LATITUDE,LONGITUDE,SPEED,TERMINALNO,TIME,TRIP_ID,Y,time,date,hour,minute
0,0,12,39.402588,41.103741,122.985168,2.15,1,1476923580,1,0.0,201610200833,20161020,8,33
1,0,24,39.311157,41.104904,122.984398,4.11,1,1476923640,1,0.0,201610200834,20161020,8,34
2,0,74,34.178955,41.106388,122.986496,2.99,1,1476923700,1,0.0,201610200835,20161020,8,35
3,0,115,37.765381,41.106884,122.989769,7.59,1,1476923760,1,0.0,201610200836,20161020,8,36
4,0,151,36.049194,41.105442,122.991089,0.24,1,1476923820,1,0.0,201610200837,20161020,8,37


In [11]:
feature.head()

,TERMINALNO,trip_max,lon_max,lon_min,lon,lat_max,lat_min,lat,heg_max,heg_min,heg_mean,heg,vol,sp_max,sp_mean,call0,call1,call_ratio_0,call_ratio_1
0,1,45,124.809792,122.971771,1.838021,43.506298,41.082325,2.423973,224.069580,-7.248444,47.848093,231.318024,1030.594264,32.779999,17.489840,144.0,150.0,0.489796,0.510204
1,2,66,117.079720,113.330002,3.749718,30.790524,22.917767,7.872757,526.300537,-112.462982,71.400718,638.763519,18856.694241,36.119999,9.287734,289.0,439.0,0.396978,0.603022
2,3,37,113.918510,112.899796,1.018714,28.856243,22.474741,6.381502,125.748291,-10.797699,43.942672,136.545990,887.675299,25.440001,7.987331,838.0,125.0,0.870197,0.129803
3,4,98,116.943901,116.751045,0.192856,33.992123,33.473557,0.518566,115.885498,-14.475311,31.659250,130.360809,13.037197,33.310001,6.312753,181.0,611.0,0.228535,0.771465
4,5,85,115.024368,114.194641,0.829727,31.188187,30.531494,0.656693,117.702576,-14.855835,29.461915,132.558411,72.227885,53.480000,7.695846,903.0,0.0,1.000000,0.000000
